# Package Imoport and Installation

In [1]:
%%capture
!pip install ultralytics
!pip install onnx
!pip install onnx_tf
!pip install matplotlib
!pip install protobuf==3.20.1
!pip install keras==2.11.0
!pip install tensorflow-estimator==2.11.0
!pip install tensorboard==2.11.0
!pip install torchinfo
!pip install torchview

In [2]:
%%capture
import os
import json, datetime
import torch
from ultralytics import YOLO
from torchinfo import summary
from yolov8_EE_network import YOLOv8n_EE #implemented network leveraging ultralytics and torch utils
from torch_graph import model_graph #implemented graph service for demonstrating model's architecture

In [ ]:
ROOT_DIR= '/content/drive/MyDrive/dataset/dataset' #our dataset's directory
test_folder= os.path.join(ROOT_DIR, 'test/images/')
DATA_YAML= os.path.join(ROOT_DIR, 'data.yaml') #dataset yaml file
MODEL_YAML = "/content/drive/MyDrive/EE using ultralitycs/yolov8n_ee.yaml"
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu' #device declearation for model traning
IMG_SIZE= 640 #size of image and input tensors
BATCH_SIZE= 16
EPOCHS= 2
class_count = 1

(Optional) **T4-GPU Availability Checking and Drive Mounting for Providing Dataset Access**

If you are running this code on a local computer or server you may pass this section.

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/EE using ultralitycs
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/EE using ultralitycs
'Copy of Copy of YOLOv8_EarlyExit.ipynb'   torch_graph.py
 __pycache__				   yolo11n.pt
 requirements.txt			   yolo_early_exit_torchview.png
 runs					   YOLOv8_EarlyExit.ipynb
 tensor.pt				   yolov8_EE_network.py
 test.ipynb				   yolov8n_ee.yaml
 torch_graph.ipynb			   yolov8n.pt


In [7]:
print("Torch Cuda - GPU informaiton:")
print("availablity: ", torch.cuda.is_available())
print("name: ", torch.cuda.get_device_name())
print("properties: ", torch.cuda.get_device_properties(device= 0))

Torch Cuda - GPU informaiton:
availablity:  True
name:  Tesla T4
properties:  _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=4bfbf75f-fa0b-7365-4635-842fa243dd30, pci_bus_id=0, pci_device_id=4, pci_domain_id=0, L2_cache_size=4MB)


# Early Exit Implementation in YOLOv8n

## Adding Early-Exit layers using ultralytics utilities

Here we will reimplement YOLOv8's architecture and will add 5 detect layers in backbone and neck aim to increase its infrence time and decrease its computational overheads while we are tending to maintaion accuracy's rates static as far as its possible.

In [46]:
#Build model and extract its summary
yolo_ee = YOLO(MODEL_YAML)

yolo_ee.model = YOLOv8n_EE(nc=class_count,model_yaml=MODEL_YAML).to(DEVICE)

summary(yolo_ee)

Layer (type:depth-idx)                             Param #
YOLO                                               --
├─YOLOv8n_EE: 1-1                                  --
│    └─Conv: 2-1                                   --
│    │    └─Conv2d: 3-1                            1,728
│    │    └─BatchNorm2d: 3-2                       128
│    │    └─SiLU: 3-3                              --
│    └─Conv: 2-2                                   --
│    │    └─Conv2d: 3-4                            73,728
│    │    └─BatchNorm2d: 3-5                       256
│    │    └─SiLU: 3-6                              --
│    └─C2f: 2-3                                    --
│    │    └─Conv: 3-7                              33,280
│    │    └─Conv: 3-8                              82,176
│    │    └─ModuleList: 3-9                        886,272
│    └─Detect: 2-4                                 --
│    │    └─ModuleList: 3-10                       115,008
│    │    └─ModuleList: 3-11                      

In [ ]:
model_graph(
    yolo_ee,
    expand_nested=False,
    graph_name="YOLO_EE_Graph",
)

In [ ]:
results = yolo_ee.train(
    data=DATA_YAML,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    epochs=EPOCHS,
    device=DEVICE,
    name='yolov8n-ee',
    pretrained=False
)


## Test and Validation

In [42]:
def log_exit(event):
    """  Access model outputs per batch; exact fields vary by version, this function will help us in analyzing early exit layer performance
         Args:
              event: event is a Predictor/Validator object depending on stage that we can extract needed statics from
    """
        # event is a Predictor/Validator object depending on stage
    #

    batch = event.batch
    preds = event.results  # or event.batch_preds / event.outputs in some versions
    now = datetime.datetime.now().isoformat(timespec='seconds')
    # You can also read a flag you set on your model during forward, e.g., model.last_exit_id
    exit_id = getattr(event.model, "outputs", None)
    print(exit_id)
    with open("exit_log.jsonl", "a") as f:
        f.write(json.dumps({"t": now, "stage": event.__class__.__name__,
                           "exit": exit_id, "batch_size": len(preds)}) + "\n")

In [ ]:
model_path = "/content/drive/MyDrive/EE using ultralitycs/runs/detect/yolov8n-ee22/weights/best.pt"
# 1) Load the Ultralytics model from the .pt
model = YOLO(model_path)

test_results = model.predict(source=test_folder, save=True, show=False)
print("Prediction complete. Check the 'runs/detect/predict/' folder for the saved results with bounding boxes.")

# Test and Valifation
model.val(data=DATA_YAML,split="test")

print("Validation complete. Check the 'runs/detect/valid/' folder for the saved validation statics.")